# Relatório Final — Clusterização de Criptomoedas por Perfil de Risco

O mercado de criptomoedas possui milhares de ativos, com níveis muito distintos de
volatilidade, risco e estabilidade. Para um investidor iniciante, é difícil identificar:

- quais moedas são mais estáveis  
- quais são mais arriscadas  
- quais têm comportamento semelhante  

### **Objetivo do Projeto**
Agrupar criptomoedas por nível de risco utilizando técnicas de **aprendizado não supervisionado**, destacando padrões de comportamento.

Isso ajuda investidores a:

- montar carteiras alinhadas ao seu perfil
- diversificar riscos
- identificar grupos de ativos com comportamento similar

# 1. Metodologia do Projeto

O projeto segue 5 etapas principais:

---

## **1. Coleta de Dados**
Via API do CoinGecko:
- preços históricos (120 dias)
- volumes
- market cap

---

## **2. Análise Exploratória (EDA)**
Com foco em:
- volatilidade
- correlações
- variações temporais
- identificação de outliers

---

## **3. Feature Engineering**
Para cada moeda, calculamos:

### Métricas financeiras
- **avg_return** → retorno médio
- **volatility** → volatilidade total
- **downside_std** → volatilidade apenas dos retornos negativos
- **abs_return** → retorno absoluto médio (medida forte de instabilidade)

### Métricas fundamentais
- **avg_volume**
- **avg_marketcap**

### Transformações aplicadas
- `log_volume` normalizado (evita dominância do BTC)
- `log_mc` (marketcap log)

### Ajuste importante
O market cap recebeu **peso multiplicativo 5x** na etapa de modelagem, pois é um dos maiores indicadores de estabilidade.

---

## **4. Modelagem**
Modelos avaliados:

- **K-Means (k=3)** → modelo escolhido  
- DBSCAN → teste exploratório

O K-Means teve melhor separação entre grupos.

---

## **5. Visualização**
Foi aplicado PCA (2 componentes) para representar os clusters em 2D.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 3. Carregando os Clusters Finais

In [ ]:
df = pd.read_csv("../data/outputs/crypto_clusters.csv", index_col="coin")
df.head()

# 4. Visualização dos Grupos
A seguir, vemos como as criptomoedas foram distribuídas no espaço reduzido (PCA).

In [ ]:
plt.figure(figsize=(10,6))

sns.scatterplot(
    x=df['pc1'],
    y=df['pc2'],
    hue=df['cluster'],
    palette='Set1',
    s=120
)

for coin, row in df.iterrows():
    plt.text(row['pc1']+0.015, row['pc2']+0.015, coin, fontsize=8)

plt.title("Clusters de Criptomoedas (PCA 2D)")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.grid(True)
plt.show()

# 5. Perfil de Cada Cluster

A tabela abaixo resume as métricas médias de cada grupo.

In [ ]:
cluster_profile = df.groupby('cluster').mean()[[
    'avg_return','volatility','downside_std','abs_return',
    'avg_volume','avg_marketcap'
]]

cluster_profile

plt.figure(figsize=(9,6))
sns.heatmap(cluster_profile, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Perfil Médio por Cluster")
plt.show()

# 6. Interpretação dos Resultados

Com base nas métricas analisadas, os clusters podem ser interpretados como:

### **Cluster 2 — Perfil Conservador**
- baixa volatilidade  
- retorno moderado  
- market cap alto  
- menor risco de queda  

### **Cluster 1 — Perfil Moderado**
- volatilidade média  
- risco de queda equilibrado  

### **Cluster 0 — Perfil Agressivo**
- maior volatilidade  
- maior risco de queda  
- normalmente market cap menor  
- comportamento mais especulativo  

Essa classificação auxilia investidores a montar carteiras coerentes com seu perfil.

# 7. Conclusões

- O modelo K-Means conseguiu separar criptomoedas em 3 grupos relevantes.
- O PCA mostrou visualmente que os clusters possuem características distintas.
- As métricas de risco foram fundamentais para distinguir comportamento.
- O projeto mostra que é possível aplicar ciência de dados para segmentação
  de ativos financeiros com dados públicos.

O próximo passo seria:
- Expandir para mais criptos
- Testar diferentes janelas de tempo
- Criar um dashboard interativo (Streamlit)

# 8. Arquivo Gerado

O arquivo final com todos os clusters está salvo em:

`data/outputs/crypto_clusters.csv`